In [1]:
import os
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

C:\Users\parmi\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))

DATA_DIR = os.path.join(PROJECT_ROOT, "data")
PRED_DIR = os.path.join(DATA_DIR, "predictions")

AQI_RESULTS_DIR = os.path.join(DATA_DIR, "aqi_calculation_results")
os.makedirs(AQI_RESULTS_DIR, exist_ok=True)

OUTPUT_FILE = os.path.join(AQI_RESULTS_DIR, "aqi_predictions.csv")

In [3]:
# CPCB BREAKPOINTS are the ranges of the labels as per standard 
BREAKPOINTS = {
    "pm25": [(0,30,0,50),(31,60,51,100),(61,90,101,200),
             (91,120,201,300),(121,250,301,400),(251,500,401,500)],
    "pm10": [(0,50,0,50),(51,100,51,100),(101,250,101,200),
             (251,350,201,300),(351,430,301,400),(431,600,401,500)],
    "no2":  [(0,40,0,50),(41,80,51,100),(81,180,101,200),
             (181,280,201,300),(281,400,301,400),(401,600,401,500)],
    "so2":  [(0,40,0,50),(41,80,51,100),(81,380,101,200),
             (381,800,201,300),(801,1600,301,400),(1601,2000,401,500)],
    "o3":   [(0,50,0,50),(51,100,51,100),(101,168,101,200),
             (169,208,201,300),(209,748,301,400),(749,1000,401,500)],
    "co":   [(0,1,0,50),(1.1,2,51,100),(2.1,10,101,200),
             (10.1,17,201,300),(17.1,34,301,400),(34.1,50,401,500)]
}

In [4]:
# Maximum realistic values for capping
MAX_VALUES = {
    "pm25": 500,
    "pm10": 600,
    "no2": 600,
    "so2": 2000,
    "o3": 1000,
    "co": 50
}


In [5]:
def calculate_subindex(cp, breakpoints):
    cp = max(0, min(cp, breakpoints[-1][1]))
    for bp_low, bp_high, i_low, i_high in breakpoints:
        if bp_low <= cp <= bp_high:
            return ((i_high - i_low) / (bp_high - bp_low)) * (cp - bp_low) + i_low
    return 0

def aqi_category(aqi):
    if aqi <= 50: return "Good"
    elif aqi <= 100: return "Satisfactory"
    elif aqi <= 200: return "Moderate"
    elif aqi <= 300: return "Poor"
    elif aqi <= 400: return "Very Poor"
    else: return "Severe"

In [6]:
dfs = []
num_days = None  # To keep track of prediction length

for pollutant in BREAKPOINTS:
    file_path = os.path.join(PRED_DIR, f"{pollutant}_predictions.csv")

    if not os.path.exists(file_path):
        print(f" Missing file: {file_path}, skipping {pollutant}")
        continue

    df = pd.read_csv(file_path)
    if df.empty:
        print(f"File {file_path} is empty, skipping {pollutant}")
        continue

    # Use predicted values and ignore old date
    df[pollutant] = df[f"{pollutant}_pred"].clip(lower=0, upper=MAX_VALUES[pollutant])
    df = df[[pollutant]]  # Keep only the predicted values

    if num_days is None:
        num_days = len(df)

    dfs.append(df)

if not dfs:
    raise ValueError(" No valid pollutant prediction files found!")

In [7]:
# date range starting from today
start_date = datetime.today().replace(hour=0, minute=0, second=0, microsecond=0)
# start_date = datetime.strptime("14-12-2025", "%d-%m-%Y")  # can also use datetime.today()
future_dates = [start_date + timedelta(days=i) for i in range(num_days)]
date_df = pd.DataFrame({"date": future_dates})

In [8]:
# Merge all pollutant predictions
aqi_df = date_df.copy()
for df, pollutant in zip(dfs, BREAKPOINTS):
    aqi_df[pollutant] = df[pollutant].values


In [9]:
for pollutant, bps in BREAKPOINTS.items():
    if pollutant in aqi_df.columns:
        aqi_df[f"{pollutant}_sub"] = aqi_df[pollutant].apply(lambda x: calculate_subindex(x, bps))
    else:
        aqi_df[f"{pollutant}_sub"] = 0

In [10]:
# Compute final AQI
sub_cols = [c for c in aqi_df.columns if c.endswith("_sub")]
aqi_df["AQI"] = aqi_df[sub_cols].max(axis=1)
aqi_df["AQI_Category"] = aqi_df["AQI"].apply(aqi_category)

In [11]:
aqi_df['date'] = aqi_df['date'].dt.strftime('%d-%m-%Y')  # DD-MM-YYYY
aqi_df.to_csv(OUTPUT_FILE, index=False)

print("Future AQI prediction completed successfully")
print(f"Saved at: {OUTPUT_FILE}")
print(f"Days written: {len(aqi_df)}")

Future AQI prediction completed successfully
Saved at: C:\Users\parmi\CDAC_DBDA_PROJECT\AQI\data\aqi_calculation_results\aqi_predictions.csv
Days written: 7


In [13]:
# import os
# import pandas as pd
# import numpy as np
# from datetime import datetime, timedelta

# # -------------------------------
# # FIXED PROJECT PATH
# # -------------------------------
# PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))

# DATA_DIR = os.path.join(PROJECT_ROOT, "data")
# PRED_DIR = os.path.join(DATA_DIR, "predictions")

# AQI_RESULTS_DIR = os.path.join(DATA_DIR, "aqi_calculation_results")
# os.makedirs(AQI_RESULTS_DIR, exist_ok=True)

# OUTPUT_FILE = os.path.join(AQI_RESULTS_DIR, "aqi_predictions.csv")

# # -------------------------------
# # CPCB BREAKPOINTS
# # -------------------------------
# BREAKPOINTS = {
#     "pm25": [(0,30,0,50),(31,60,51,100),(61,90,101,200),
#              (91,120,201,300),(121,250,301,400),(251,500,401,500)],
#     "pm10": [(0,50,0,50),(51,100,51,100),(101,250,101,200),
#              (251,350,201,300),(351,430,301,400),(431,600,401,500)],
#     "no2":  [(0,40,0,50),(41,80,51,100),(81,180,101,200),
#              (181,280,201,300),(281,400,301,400),(401,600,401,500)],
#     "so2":  [(0,40,0,50),(41,80,51,100),(81,380,101,200),
#              (381,800,201,300),(801,1600,301,400),(1601,2000,401,500)],
#     "o3":   [(0,50,0,50),(51,100,51,100),(101,168,101,200),
#              (169,208,201,300),(209,748,301,400),(749,1000,401,500)],
#     "co":   [(0,1,0,50),(1.1,2,51,100),(2.1,10,101,200),
#              (10.1,17,201,300),(17.1,34,301,400),(34.1,50,401,500)]
# }

# # Maximum realistic values for capping
# MAX_VALUES = {
#     "pm25": 500,
#     "pm10": 600,
#     "no2": 600,
#     "so2": 2000,
#     "o3": 1000,
#     "co": 50
# }

# # -------------------------------
# # AQI FUNCTIONS
# # -------------------------------
# def calculate_subindex(cp, breakpoints):
#     cp = max(0, min(cp, breakpoints[-1][1]))
#     for bp_low, bp_high, i_low, i_high in breakpoints:
#         if bp_low <= cp <= bp_high:
#             return ((i_high - i_low) / (bp_high - bp_low)) * (cp - bp_low) + i_low
#     return 0

# def aqi_category(aqi):
#     if aqi <= 50: return "Good"
#     elif aqi <= 100: return "Satisfactory"
#     elif aqi <= 200: return "Moderate"
#     elif aqi <= 300: return "Poor"
#     elif aqi <= 400: return "Very Poor"
#     else: return "Severe"

# # -------------------------------
# # LOAD & MERGE PREDICTIONS
# # -------------------------------
# dfs = []
# num_days = None  # To keep track of prediction length

# for pollutant in BREAKPOINTS:
#     file_path = os.path.join(PRED_DIR, f"{pollutant}_predictions.csv")

#     if not os.path.exists(file_path):
#         print(f" Missing file: {file_path}, skipping {pollutant}")
#         continue

#     df = pd.read_csv(file_path)
#     if df.empty:
#         print(f"File {file_path} is empty, skipping {pollutant}")
#         continue

#     # Use predicted values and ignore old date
#     df[pollutant] = df[f"{pollutant}_pred"].clip(lower=0, upper=MAX_VALUES[pollutant])
#     df = df[[pollutant]]  # Keep only the predicted values

#     if num_days is None:
#         num_days = len(df)

#     dfs.append(df)

# if not dfs:
#     raise ValueError(" No valid pollutant prediction files found!")

# # Create a date range starting from today (or fixed date)
# start_date = datetime.today().replace(hour=0, minute=0, second=0, microsecond=0)
# # start_date = datetime.strptime("14-12-2025", "%d-%m-%Y")  # can also use datetime.today()
# future_dates = [start_date + timedelta(days=i) for i in range(num_days)]
# date_df = pd.DataFrame({"date": future_dates})

# # Merge all pollutant predictions
# aqi_df = date_df.copy()
# for df, pollutant in zip(dfs, BREAKPOINTS):
#     aqi_df[pollutant] = df[pollutant].values

# # -------------------------------
# # AQI CALCULATION
# # -------------------------------
# for pollutant, bps in BREAKPOINTS.items():
#     if pollutant in aqi_df.columns:
#         aqi_df[f"{pollutant}_sub"] = aqi_df[pollutant].apply(lambda x: calculate_subindex(x, bps))
#     else:
#         aqi_df[f"{pollutant}_sub"] = 0

# # Compute final AQI
# sub_cols = [c for c in aqi_df.columns if c.endswith("_sub")]
# aqi_df["AQI"] = aqi_df[sub_cols].max(axis=1)
# aqi_df["AQI_Category"] = aqi_df["AQI"].apply(aqi_category)

# # -------------------------------
# # FORMAT DATE & SAVE
# # -------------------------------
# aqi_df['date'] = aqi_df['date'].dt.strftime('%d-%m-%Y')  # DD-MM-YYYY
# aqi_df.to_csv(OUTPUT_FILE, index=False)

# print("✅ Future AQI prediction completed successfully")
# print(f"📁 Saved at: {OUTPUT_FILE}")
# print(f"📊 Days written: {len(aqi_df)}")


✅ Future AQI prediction completed successfully
📁 Saved at: C:\Users\parmi\CDAC_DBDA_PROJECT\AQI\data\aqi_calculation_results\aqi_predictions.csv
📊 Days written: 7


In [19]:
# import os
# import pandas as pd

# # -------------------------------
# # FIXED PROJECT PATH
# # -------------------------------
# PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))

# DATA_DIR = os.path.join(PROJECT_ROOT, "data")
# PRED_DIR = os.path.join(DATA_DIR, "predictions")

# AQI_RESULTS_DIR = os.path.join(DATA_DIR, "aqi_calculation_results")
# os.makedirs(AQI_RESULTS_DIR, exist_ok=True)

# OUTPUT_FILE = os.path.join(AQI_RESULTS_DIR, "aqi_predictions.csv")

# # -------------------------------
# # CPCB BREAKPOINTS
# # -------------------------------
# BREAKPOINTS = {
#     "pm25": [(0,30,0,50),(31,60,51,100),(61,90,101,200),
#              (91,120,201,300),(121,250,301,400),(251,500,401,500)],
#     "pm10": [(0,50,0,50),(51,100,51,100),(101,250,101,200),
#              (251,350,201,300),(351,430,301,400),(431,600,401,500)],
#     "no2":  [(0,40,0,50),(41,80,51,100),(81,180,101,200),
#              (181,280,201,300),(281,400,301,400),(401,600,401,500)],
#     "so2":  [(0,40,0,50),(41,80,51,100),(81,380,101,200),
#              (381,800,201,300),(801,1600,301,400),(1601,2000,401,500)],
#     "o3":   [(0,50,0,50),(51,100,51,100),(101,168,101,200),
#              (169,208,201,300),(209,748,301,400),(749,1000,401,500)],
#     "co":   [(0,1,0,50),(1.1,2,51,100),(2.1,10,101,200),
#              (10.1,17,201,300),(17.1,34,301,400),(34.1,50,401,500)]
# }

# # Maximum values for capping
# MAX_VALUES = {
#     "pm25": 500,
#     "pm10": 600,
#     "no2": 600,
#     "so2": 2000,
#     "o3": 1000,
#     "co": 50
# }

# # -------------------------------
# # AQI FUNCTIONS
# # -------------------------------
# def calculate_subindex(cp, breakpoints):
#     # Cap the concentration to prevent extreme AQI
#     cp = min(cp, breakpoints[-1][1])
#     for bp_low, bp_high, i_low, i_high in breakpoints:
#         if bp_low <= cp <= bp_high:
#             return ((i_high - i_low) / (bp_high - bp_low)) * (cp - bp_low) + i_low
#     return None

# def aqi_category(aqi):
#     if aqi <= 50: return "Good"
#     elif aqi <= 100: return "Satisfactory"
#     elif aqi <= 200: return "Moderate"
#     elif aqi <= 300: return "Poor"
#     elif aqi <= 400: return "Very Poor"
#     else: return "Severe"

# # -------------------------------
# # LOAD & MERGE PREDICTIONS
# # -------------------------------
# dfs = []

# for pollutant in BREAKPOINTS:
#     file_path = os.path.join(PRED_DIR, f"{pollutant}_predictions.csv")

#     if not os.path.exists(file_path):
#         print(f"Missing file: {file_path}, skipping {pollutant}")
#         continue

#     df = pd.read_csv(file_path)
#     if df.empty:
#         print(f" File {file_path} is empty, skipping {pollutant}")
#         continue

#     # Convert to UTC then naive datetime
#     df["date"] = pd.to_datetime(df["date"], utc=True).dt.tz_convert(None)
    
#     # Aggregate to daily averages
#     df["date"] = df["date"].dt.date
#     df = df.groupby("date").mean().reset_index()

#     # Cap values at max
#     max_val = MAX_VALUES[pollutant]
#     df[pollutant] = df[f"{pollutant}_pred"].clip(upper=max_val)

#     # Rename column
#     df = df.drop(columns=[f"{pollutant}_pred"])
#     dfs.append(df)

# if not dfs:
#     raise ValueError(" No valid pollutant prediction files found!")

# # Merge on date
# aqi_df = dfs[0]
# for df in dfs[1:]:
#     aqi_df = aqi_df.merge(df, on="date", how="outer")

# aqi_df = aqi_df.sort_values("date").reset_index(drop=True)

# # -------------------------------
# # AQI CALCULATION
# # -------------------------------
# for pollutant, bps in BREAKPOINTS.items():
#     if pollutant in aqi_df.columns:
#         aqi_df[f"{pollutant}_sub"] = aqi_df[pollutant].apply(lambda x: calculate_subindex(x, bps))
#     else:
#         aqi_df[f"{pollutant}_sub"] = None

# # Final AQI per day
# sub_cols = [c for c in aqi_df.columns if c.endswith("_sub")]
# aqi_df["AQI"] = aqi_df[sub_cols].max(axis=1)
# aqi_df["AQI_Category"] = aqi_df["AQI"].apply(aqi_category)

# # -------------------------------
# # SAVE OUTPUT
# # -------------------------------
# aqi_df.to_csv(OUTPUT_FILE, index=False)

# print("Daily-aggregated AQI calculation completed successfully")
# print(f"Saved at: {OUTPUT_FILE}")
# print(f"Days written: {len(aqi_df)}")


✅ Daily-aggregated AQI calculation completed successfully
📁 Saved at: C:\Users\parmi\CDAC_DBDA_PROJECT\AQI\data\aqi_calculation_results\aqi_predictions.csv
📊 Days written: 7


In [27]:
# import os
# import pandas as pd
# import numpy as np
# from datetime import datetime, timedelta

# # -------------------------------
# # FIXED PROJECT PATH
# # -------------------------------
# PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))

# DATA_DIR = os.path.join(PROJECT_ROOT, "data")
# PRED_DIR = os.path.join(DATA_DIR, "predictions")

# AQI_RESULTS_DIR = os.path.join(DATA_DIR, "aqi_calculation_results")
# os.makedirs(AQI_RESULTS_DIR, exist_ok=True)

# OUTPUT_FILE = os.path.join(AQI_RESULTS_DIR, "aqi_predictions.csv")

# # -------------------------------
# # CPCB BREAKPOINTS
# # -------------------------------
# BREAKPOINTS = {
#     "pm25": [(0,30,0,50),(31,60,51,100),(61,90,101,200),
#              (91,120,201,300),(121,250,301,400),(251,500,401,500)],
#     "pm10": [(0,50,0,50),(51,100,51,100),(101,250,101,200),
#              (251,350,201,300),(351,430,301,400),(431,600,401,500)],
#     "no2":  [(0,40,0,50),(41,80,51,100),(81,180,101,200),
#              (181,280,201,300),(281,400,301,400),(401,600,401,500)],
#     "so2":  [(0,40,0,50),(41,80,51,100),(81,380,101,200),
#              (381,800,201,300),(801,1600,301,400),(1601,2000,401,500)],
#     "o3":   [(0,50,0,50),(51,100,51,100),(101,168,101,200),
#              (169,208,201,300),(209,748,301,400),(749,1000,401,500)],
#     "co":   [(0,1,0,50),(1.1,2,51,100),(2.1,10,101,200),
#              (10.1,17,201,300),(17.1,34,301,400),(34.1,50,401,500)]
# }

# # Maximum realistic values for capping
# MAX_VALUES = {
#     "pm25": 500,
#     "pm10": 600,
#     "no2": 600,
#     "so2": 2000,
#     "o3": 1000,
#     "co": 50
# }

# # -------------------------------
# # AQI FUNCTIONS
# # -------------------------------
# def calculate_subindex(cp, breakpoints):
#     cp = max(0, min(cp, breakpoints[-1][1]))
#     for bp_low, bp_high, i_low, i_high in breakpoints:
#         if bp_low <= cp <= bp_high:
#             return ((i_high - i_low) / (bp_high - bp_low)) * (cp - bp_low) + i_low
#     return 0

# def aqi_category(aqi):
#     if aqi <= 50: return "Good"
#     elif aqi <= 100: return "Satisfactory"
#     elif aqi <= 200: return "Moderate"
#     elif aqi <= 300: return "Poor"
#     elif aqi <= 400: return "Very Poor"
#     else: return "Severe"

# # -------------------------------
# # LOAD & MERGE PREDICTIONS
# # -------------------------------
# dfs = []
# num_days = None  # To keep track of prediction length

# for pollutant in BREAKPOINTS:
#     file_path = os.path.join(PRED_DIR, f"{pollutant}_predictions.csv")

#     if not os.path.exists(file_path):
#         print(f"⚠️ Missing file: {file_path}, skipping {pollutant}")
#         continue

#     df = pd.read_csv(file_path)
#     if df.empty:
#         print(f"⚠️ File {file_path} is empty, skipping {pollutant}")
#         continue

#     # Use predicted values and ignore old date
#     df[pollutant] = df[f"{pollutant}_pred"].clip(lower=0, upper=MAX_VALUES[pollutant])
#     df = df[[pollutant]]  # Keep only the predicted values

#     if num_days is None:
#         num_days = len(df)

#     dfs.append(df)

# if not dfs:
#     raise ValueError("❌ No valid pollutant prediction files found!")

# # Create a date range starting from today (or fixed date)
# start_date = datetime.strptime("14-12-2025", "%d-%m-%Y")  # can also use datetime.today()
# future_dates = [start_date + timedelta(days=i) for i in range(num_days)]
# date_df = pd.DataFrame({"date": future_dates})

# # Merge all pollutant predictions
# aqi_df = date_df.copy()
# for df, pollutant in zip(dfs, BREAKPOINTS):
#     aqi_df[pollutant] = df[pollutant].values

# # -------------------------------
# # AQI CALCULATION
# # -------------------------------
# for pollutant, bps in BREAKPOINTS.items():
#     if pollutant in aqi_df.columns:
#         aqi_df[f"{pollutant}_sub"] = aqi_df[pollutant].apply(lambda x: calculate_subindex(x, bps))
#     else:
#         aqi_df[f"{pollutant}_sub"] = 0

# # Compute final AQI
# sub_cols = [c for c in aqi_df.columns if c.endswith("_sub")]
# aqi_df["AQI"] = aqi_df[sub_cols].max(axis=1)
# aqi_df["AQI_Category"] = aqi_df["AQI"].apply(aqi_category)

# # -------------------------------
# # FORMAT DATE & SAVE
# # -------------------------------
# aqi_df['date'] = aqi_df['date'].dt.strftime('%d-%m-%Y')  # DD-MM-YYYY
# aqi_df.to_csv(OUTPUT_FILE, index=False)

# print("✅ Future AQI prediction completed successfully")
# print(f"📁 Saved at: {OUTPUT_FILE}")
# print(f"📊 Days written: {len(aqi_df)}")


✅ Future AQI prediction completed successfully
📁 Saved at: C:\Users\parmi\CDAC_DBDA_PROJECT\AQI\data\aqi_calculation_results\aqi_predictions.csv
📊 Days written: 7


In [21]:
# import os
# import pandas as pd
# import numpy as np

# # -------------------------------
# # FIXED PROJECT PATH
# # -------------------------------
# PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))
# DATA_DIR = os.path.join(PROJECT_ROOT, "data")
# PRED_DIR = os.path.join(DATA_DIR, "predictions")
# PROCESSED_DIR = os.path.join(DATA_DIR, "processed")

# AQI_RESULTS_DIR = os.path.join(DATA_DIR, "aqi_calculation_results")
# os.makedirs(AQI_RESULTS_DIR, exist_ok=True)
# OUTPUT_FILE = os.path.join(AQI_RESULTS_DIR, "aqi_predictions.csv")

# # -------------------------------
# # CPCB BREAKPOINTS & MAX VALUES
# # -------------------------------
# BREAKPOINTS = {
#     "pm25": [(0,30,0,50),(31,60,51,100),(61,90,101,200),
#              (91,120,201,300),(121,250,301,400),(251,500,401,500)],
#     "pm10": [(0,50,0,50),(51,100,51,100),(101,250,101,200),
#              (251,350,201,300),(351,430,301,400),(431,600,401,500)],
#     "no2":  [(0,40,0,50),(41,80,51,100),(81,180,101,200),
#              (181,280,201,300),(281,400,301,400),(401,600,401,500)],
#     "so2":  [(0,40,0,50),(41,80,51,100),(81,380,101,200),
#              (381,800,201,300),(801,1600,301,400),(1601,2000,401,500)],
#     "o3":   [(0,50,0,50),(51,100,51,100),(101,168,101,200),
#              (169,208,201,300),(209,748,301,400),(749,1000,401,500)],
#     "co":   [(0,1,0,50),(1.1,2,51,100),(2.1,10,101,200),
#              (10.1,17,201,300),(17.1,34,301,400),(34.1,50,401,500)]
# }

# MAX_VALUES = {
#     "pm25": 500, "pm10": 600, "no2": 600,
#     "so2": 2000, "o3": 1000, "co": 50
# }

# # -------------------------------
# # AQI FUNCTIONS
# # -------------------------------
# def calculate_subindex(cp, breakpoints):
#     cp = max(0, min(cp, breakpoints[-1][1]))  # clip to [0, max]
#     for bp_low, bp_high, i_low, i_high in breakpoints:
#         if bp_low <= cp <= bp_high:
#             return ((i_high - i_low)/(bp_high - bp_low))*(cp - bp_low) + i_low
#     return None

# def aqi_category(aqi):
#     if aqi <= 50: return "Good"
#     elif aqi <= 100: return "Satisfactory"
#     elif aqi <= 200: return "Moderate"
#     elif aqi <= 300: return "Poor"
#     elif aqi <= 400: return "Very Poor"
#     else: return "Severe"

# # -------------------------------
# # LOAD & MERGE PREDICTIONS
# # -------------------------------
# dfs = []

# for pollutant in BREAKPOINTS:
#     pred_file = os.path.join(PRED_DIR, f"{pollutant}_predictions.csv")
#     minmax_file = os.path.join(PROCESSED_DIR, f"{pollutant}_minmax.csv")
    
#     if not os.path.exists(pred_file) or not os.path.exists(minmax_file):
#         print(f"⚠️ Missing prediction or min/max file for {pollutant}, skipping.")
#         continue
    
#     df = pd.read_csv(pred_file)
#     minmax_df = pd.read_csv(minmax_file)
    
#     # Un-normalize using training min/max
#     min_val = minmax_df['min'].iloc[0]
#     max_val = minmax_df['max'].iloc[0]
#     df[pollutant] = df[f"{pollutant}_pred"] * (max_val - min_val) + min_val
    
#     # Clip values to realistic max
#     df[pollutant] = df[pollutant].clip(lower=0, upper=MAX_VALUES[pollutant])
    
#     # Convert datetime to date
#     df["date"] = pd.to_datetime(df["date"]).dt.date
#     df = df.groupby("date").mean().reset_index()
    
#     df = df[["date", pollutant]]
#     dfs.append(df)

# if not dfs:
#     raise ValueError("❌ No valid prediction files found!")

# # Merge all pollutants on date
# aqi_df = dfs[0]
# for df in dfs[1:]:
#     aqi_df = aqi_df.merge(df, on="date", how="outer")

# aqi_df = aqi_df.sort_values("date").reset_index(drop=True)

# # -------------------------------
# # CALCULATE AQI
# # -------------------------------
# for pollutant, bps in BREAKPOINTS.items():
#     if pollutant in aqi_df.columns:
#         aqi_df[f"{pollutant}_sub"] = aqi_df[pollutant].apply(lambda x: calculate_subindex(x, bps))

# # Final AQI
# sub_cols = [c for c in aqi_df.columns if c.endswith("_sub")]
# aqi_df["AQI"] = aqi_df[sub_cols].max(axis=1)
# aqi_df["AQI_Category"] = aqi_df["AQI"].apply(aqi_category)

# # -------------------------------
# # SAVE OUTPUT
# # -------------------------------
# aqi_df.to_csv(OUTPUT_FILE, index=False)

# print("✅ Corrected AQI calculation completed")
# print(f"📁 Saved at: {OUTPUT_FILE}")
# print(f"📊 Days written: {len(aqi_df)}")
# # gives final ouput as severe

✅ Corrected AQI calculation completed
📁 Saved at: C:\Users\parmi\CDAC_DBDA_PROJECT\AQI\data\aqi_calculation_results\aqi_predictions.csv
📊 Days written: 7


In [15]:
# import os
# import pandas as pd

# # -------------------------------
# # FIXED PROJECT PATH
# # -------------------------------
# PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))

# DATA_DIR = os.path.join(PROJECT_ROOT, "data")
# PRED_DIR = os.path.join(DATA_DIR, "predictions")

# AQI_RESULTS_DIR = os.path.join(DATA_DIR, "aqi_calculation_results")
# os.makedirs(AQI_RESULTS_DIR, exist_ok=True)

# OUTPUT_FILE = os.path.join(AQI_RESULTS_DIR, "aqi_predictions.csv")

# # -------------------------------
# # CPCB BREAKPOINTS
# # -------------------------------
# BREAKPOINTS = {
#     "pm25": [(0,30,0,50),(31,60,51,100),(61,90,101,200),
#              (91,120,201,300),(121,250,301,400),(251,500,401,500)],
#     "pm10": [(0,50,0,50),(51,100,51,100),(101,250,101,200),
#              (251,350,201,300),(351,430,301,400),(431,600,401,500)],
#     "no2":  [(0,40,0,50),(41,80,51,100),(81,180,101,200),
#              (181,280,201,300),(281,400,301,400),(401,600,401,500)],
#     "so2":  [(0,40,0,50),(41,80,51,100),(81,380,101,200),
#              (381,800,201,300),(801,1600,301,400),(1601,2000,401,500)],
#     "o3":   [(0,50,0,50),(51,100,51,100),(101,168,101,200),
#              (169,208,201,300),(209,748,301,400),(749,1000,401,500)],
#     "co":   [(0,1,0,50),(1.1,2,51,100),(2.1,10,101,200),
#              (10.1,17,201,300),(17.1,34,301,400),(34.1,50,401,500)]
# }

# # -------------------------------
# # AQI FUNCTIONS
# # -------------------------------
# def calculate_subindex(cp, breakpoints):
#     for bp_low, bp_high, i_low, i_high in breakpoints:
#         if bp_low <= cp <= bp_high:
#             return ((i_high - i_low) / (bp_high - bp_low)) * (cp - bp_low) + i_low
#     return None

# def aqi_category(aqi):
#     if aqi <= 50: return "Good"
#     elif aqi <= 100: return "Satisfactory"
#     elif aqi <= 200: return "Moderate"
#     elif aqi <= 300: return "Poor"
#     elif aqi <= 400: return "Very Poor"
#     else: return "Severe"

# # -------------------------------
# # LOAD & MERGE PREDICTIONS
# # -------------------------------
# dfs = []

# for pollutant in BREAKPOINTS:
#     file_path = os.path.join(PRED_DIR, f"{pollutant}_predictions.csv")

#     if not os.path.exists(file_path):
#         print(f"⚠️  Missing file: {file_path}, skipping {pollutant}")
#         continue

#     df = pd.read_csv(file_path)

#     if df.empty:
#         print(f"⚠️  File {file_path} is empty, skipping {pollutant}")
#         continue

#     # Convert to UTC then naive datetime
#     df["date"] = pd.to_datetime(df["date"], utc=True).dt.tz_convert(None)
    
#     # Aggregate to DAILY averages
#     df["date"] = df["date"].dt.date  # only keep date
#     df = df.groupby("date").mean().reset_index()  # average if multiple rows per day

#     # Rename prediction column
#     df = df.rename(columns={f"{pollutant}_pred": pollutant})

#     dfs.append(df)

# if not dfs:
#     raise ValueError("❌ No valid pollutant prediction files found!")

# # Merge on date
# aqi_df = dfs[0]
# for df in dfs[1:]:
#     aqi_df = aqi_df.merge(df, on="date", how="outer")

# aqi_df = aqi_df.sort_values("date").reset_index(drop=True)

# # -------------------------------
# # AQI CALCULATION
# # -------------------------------
# for pollutant, bps in BREAKPOINTS.items():
#     if pollutant in aqi_df.columns:
#         aqi_df[f"{pollutant}_sub"] = aqi_df[pollutant].apply(lambda x: calculate_subindex(x, bps))
#     else:
#         aqi_df[f"{pollutant}_sub"] = None

# # Calculate final AQI per day
# sub_cols = [c for c in aqi_df.columns if c.endswith("_sub")]
# aqi_df["AQI"] = aqi_df[sub_cols].max(axis=1)
# aqi_df["AQI_Category"] = aqi_df["AQI"].apply(aqi_category)

# # -------------------------------
# # SAVE OUTPUT
# # -------------------------------
# aqi_df.to_csv(OUTPUT_FILE, index=False)

# print("✅ Daily-aggregated AQI calculation completed successfully")
# print(f"📁 Saved at: {OUTPUT_FILE}")
# print(f"📊 Days written: {len(aqi_df)}")

# '''
# ✅ Daily-aggregated AQI calculation completed successfully
# 📁 Saved at: C:\Users\parmi\CDAC_DBDA_PROJECT\AQI\data\aqi_calculation_results\aqi_predictions.csv
# 📊 Days written: 7
# '''


✅ Daily-aggregated AQI calculation completed successfully
📁 Saved at: C:\Users\parmi\CDAC_DBDA_PROJECT\AQI\data\aqi_calculation_results\aqi_predictions.csv
📊 Days written: 7


In [16]:
# import os
# import pandas as pd

# # -------------------------------
# # FIXED PROJECT PATH
# # -------------------------------
# PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))

# DATA_DIR = os.path.join(PROJECT_ROOT, "data")
# PRED_DIR = os.path.join(DATA_DIR, "predictions")

# AQI_RESULTS_DIR = os.path.join(DATA_DIR, "aqi_calculation_results")
# os.makedirs(AQI_RESULTS_DIR, exist_ok=True)

# OUTPUT_FILE = os.path.join(AQI_RESULTS_DIR, "aqi_predictions.csv")

# # -------------------------------
# # CPCB BREAKPOINTS
# # -------------------------------
# BREAKPOINTS = {
#     "pm25": [(0,30,0,50),(31,60,51,100),(61,90,101,200),
#              (91,120,201,300),(121,250,301,400),(251,500,401,500)],
#     "pm10": [(0,50,0,50),(51,100,51,100),(101,250,101,200),
#              (251,350,201,300),(351,430,301,400),(431,600,401,500)],
#     "no2":  [(0,40,0,50),(41,80,51,100),(81,180,101,200),
#              (181,280,201,300),(281,400,301,400),(401,600,401,500)],
#     "so2":  [(0,40,0,50),(41,80,51,100),(81,380,101,200),
#              (381,800,201,300),(801,1600,301,400),(1601,2000,401,500)],
#     "o3":   [(0,50,0,50),(51,100,51,100),(101,168,101,200),
#              (169,208,201,300),(209,748,301,400),(749,1000,401,500)],
#     "co":   [(0,1,0,50),(1.1,2,51,100),(2.1,10,101,200),
#              (10.1,17,201,300),(17.1,34,301,400),(34.1,50,401,500)]
# }

# # -------------------------------
# # MIN/MAX VALUES FOR RESCALING (from training data)
# # Replace these with your actual min/max values
# # If predictions are already in μg/m3, keep min=max=1
# # -------------------------------
# SCALER_PARAMS = {
#     "pm25": {"min": 0, "max": 500},
#     "pm10": {"min": 0, "max": 600},
#     "no2": {"min": 0, "max": 600},
#     "so2": {"min": 0, "max": 2000},
#     "o3": {"min": 0, "max": 1000},
#     "co": {"min": 0, "max": 50}
# }

# # -------------------------------
# # AQI FUNCTIONS
# # -------------------------------
# def calculate_subindex(cp, breakpoints):
#     for bp_low, bp_high, i_low, i_high in breakpoints:
#         if bp_low <= cp <= bp_high:
#             return ((i_high - i_low) / (bp_high - bp_low)) * (cp - bp_low) + i_low
#     return None

# def aqi_category(aqi):
#     if aqi <= 50: return "Good"
#     elif aqi <= 100: return "Satisfactory"
#     elif aqi <= 200: return "Moderate"
#     elif aqi <= 300: return "Poor"
#     elif aqi <= 400: return "Very Poor"
#     else: return "Severe"

# # -------------------------------
# # LOAD & MERGE PREDICTIONS
# # -------------------------------
# dfs = []

# for pollutant in BREAKPOINTS:
#     file_path = os.path.join(PRED_DIR, f"{pollutant}_predictions.csv")

#     if not os.path.exists(file_path):
#         print(f"⚠️ Missing file: {file_path}, skipping {pollutant}")
#         continue

#     df = pd.read_csv(file_path)

#     if df.empty:
#         print(f"⚠️ File {file_path} is empty, skipping {pollutant}")
#         continue

#     # Convert to UTC then naive datetime
#     df["date"] = pd.to_datetime(df["date"], utc=True).dt.tz_convert(None)
    
#     # Aggregate to daily averages
#     df["date"] = df["date"].dt.date
#     df = df.groupby("date").mean().reset_index()

#     # Rescale predictions if necessary
#     min_val = SCALER_PARAMS[pollutant]["min"]
#     max_val = SCALER_PARAMS[pollutant]["max"]
#     df[pollutant] = df[f"{pollutant}_pred"] * (max_val - min_val) + min_val

#     # Rename column
#     df = df.drop(columns=[f"{pollutant}_pred"])

#     dfs.append(df)

# if not dfs:
#     raise ValueError("❌ No valid pollutant prediction files found!")

# # Merge on date
# aqi_df = dfs[0]
# for df in dfs[1:]:
#     aqi_df = aqi_df.merge(df, on="date", how="outer")

# aqi_df = aqi_df.sort_values("date").reset_index(drop=True)

# # -------------------------------
# # AQI CALCULATION
# # -------------------------------
# for pollutant, bps in BREAKPOINTS.items():
#     if pollutant in aqi_df.columns:
#         aqi_df[f"{pollutant}_sub"] = aqi_df[pollutant].apply(lambda x: calculate_subindex(x, bps))
#     else:
#         aqi_df[f"{pollutant}_sub"] = None

# # Final AQI per day
# sub_cols = [c for c in aqi_df.columns if c.endswith("_sub")]
# aqi_df["AQI"] = aqi_df[sub_cols].max(axis=1)
# aqi_df["AQI_Category"] = aqi_df["AQI"].apply(aqi_category)

# # -------------------------------
# # WARN IF AQI VERY LOW
# # -------------------------------
# if (aqi_df["AQI"] < 20).all():
#     print("⚠️ Warning: All AQI values are very low (<20). Check if predictions are scaled or realistic!")

# # -------------------------------
# # SAVE OUTPUT
# # -------------------------------
# aqi_df.to_csv(OUTPUT_FILE, index=False)

# print("✅ Daily-aggregated AQI calculation completed successfully")
# print(f"📁 Saved at: {OUTPUT_FILE}")
# print(f"📊 Days written: {len(aqi_df)}")


✅ Daily-aggregated AQI calculation completed successfully
📁 Saved at: C:\Users\parmi\CDAC_DBDA_PROJECT\AQI\data\aqi_calculation_results\aqi_predictions.csv
📊 Days written: 7


In [12]:
# import os
# import pandas as pd

# # -------------------------------
# # FIXED PROJECT PATH
# # -------------------------------
# PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))

# DATA_DIR = os.path.join(PROJECT_ROOT, "data")
# PRED_DIR = os.path.join(DATA_DIR, "predictions")

# AQI_RESULTS_DIR = os.path.join(DATA_DIR, "aqi_calculation_results")
# os.makedirs(AQI_RESULTS_DIR, exist_ok=True)

# OUTPUT_FILE = os.path.join(AQI_RESULTS_DIR, "aqi_predictions.csv")

# # -------------------------------
# # CPCB BREAKPOINTS
# # -------------------------------
# BREAKPOINTS = {
#     "pm25": [(0,30,0,50),(31,60,51,100),(61,90,101,200),
#              (91,120,201,300),(121,250,301,400),(251,500,401,500)],
#     "pm10": [(0,50,0,50),(51,100,51,100),(101,250,101,200),
#              (251,350,201,300),(351,430,301,400),(431,600,401,500)],
#     "no2":  [(0,40,0,50),(41,80,51,100),(81,180,101,200),
#              (181,280,201,300),(281,400,301,400),(401,600,401,500)],
#     "so2":  [(0,40,0,50),(41,80,51,100),(81,380,101,200),
#              (381,800,201,300),(801,1600,301,400),(1601,2000,401,500)],
#     "o3":   [(0,50,0,50),(51,100,51,100),(101,168,101,200),
#              (169,208,201,300),(209,748,301,400),(749,1000,401,500)],
#     "co":   [(0,1,0,50),(1.1,2,51,100),(2.1,10,101,200),
#              (10.1,17,201,300),(17.1,34,301,400),(34.1,50,401,500)]
# }

# # -------------------------------
# # AQI FUNCTIONS
# # -------------------------------
# def calculate_subindex(cp, breakpoints):
#     for bp_low, bp_high, i_low, i_high in breakpoints:
#         if bp_low <= cp <= bp_high:
#             return ((i_high - i_low) / (bp_high - bp_low)) * (cp - bp_low) + i_low
#     return None

# def aqi_category(aqi):
#     if aqi <= 50: return "Good"
#     elif aqi <= 100: return "Satisfactory"
#     elif aqi <= 200: return "Moderate"
#     elif aqi <= 300: return "Poor"
#     elif aqi <= 400: return "Very Poor"
#     else: return "Severe"

# # -------------------------------
# # LOAD & MERGE PREDICTIONS
# # -------------------------------
# dfs = []

# for pollutant in BREAKPOINTS:
#     file_path = os.path.join(PRED_DIR, f"{pollutant}_predictions.csv")

#     if not os.path.exists(file_path):
#         print(f"⚠️  Missing file: {file_path}, skipping {pollutant}")
#         continue

#     df = pd.read_csv(file_path)

#     if df.empty:
#         print(f"⚠️  File {file_path} is empty, skipping {pollutant}")
#         continue

#     # Parse datetime as UTC and convert to naive
#     df["date"] = pd.to_datetime(df["date"], utc=True).dt.tz_convert(None)
    
#     # ROUND timestamps to nearest hour to avoid small mismatches
#     df["date"] = df["date"].dt.round("H")

#     # Rename prediction column to pollutant name
#     df = df.rename(columns={f"{pollutant}_pred": pollutant})

#     dfs.append(df)

# if not dfs:
#     raise ValueError("❌ No valid pollutant prediction files found!")

# # Merge on date (robust with rounded timestamps)
# aqi_df = dfs[0]
# for df in dfs[1:]:
#     aqi_df = aqi_df.merge(df, on="date", how="outer")  # outer merge keeps all dates

# aqi_df = aqi_df.sort_values("date").reset_index(drop=True)

# # -------------------------------
# # AQI CALCULATION
# # -------------------------------
# for pollutant, bps in BREAKPOINTS.items():
#     if pollutant in aqi_df.columns:
#         aqi_df[f"{pollutant}_sub"] = aqi_df[pollutant].apply(lambda x: calculate_subindex(x, bps))
#     else:
#         aqi_df[f"{pollutant}_sub"] = None

# # Calculate final AQI
# sub_cols = [c for c in aqi_df.columns if c.endswith("_sub")]
# aqi_df["AQI"] = aqi_df[sub_cols].max(axis=1)
# aqi_df["AQI_Category"] = aqi_df["AQI"].apply(aqi_category)

# # -------------------------------
# # SAVE OUTPUT
# # -------------------------------
# aqi_df.to_csv(OUTPUT_FILE, index=False)

# print("✅ AQI calculation completed successfully")
# print(f"📁 Saved at: {OUTPUT_FILE}")
# print(f"📊 Rows written: {len(aqi_df)}")


✅ AQI calculation completed successfully
📁 Saved at: C:\Users\parmi\CDAC_DBDA_PROJECT\AQI\data\aqi_calculation_results\aqi_predictions.csv
📊 Rows written: 7


C:\Users\parmi\AppData\Local\Temp\ipykernel_20292\748103218.py:74: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df["date"] = df["date"].dt.round("H")
C:\Users\parmi\AppData\Local\Temp\ipykernel_20292\748103218.py:74: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df["date"] = df["date"].dt.round("H")
C:\Users\parmi\AppData\Local\Temp\ipykernel_20292\748103218.py:74: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df["date"] = df["date"].dt.round("H")
C:\Users\parmi\AppData\Local\Temp\ipykernel_20292\748103218.py:74: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df["date"] = df["date"].dt.round("H")
C:\Users\parmi\AppData\Local\Temp\ipykernel_20292\748103218.py:74: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df["date"] = 

In [5]:
# import os
# import pandas as pd
# import numpy as np

# # -------------------------------
# # PATHS
# # -------------------------------
# PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))

# DATA_DIR = os.path.join(PROJECT_ROOT, "data")
# PRED_DIR = os.path.join(DATA_DIR, "predictions")

# AQI_RESULTS_DIR = os.path.join(DATA_DIR, "aqi_calculation_results")
# os.makedirs(AQI_RESULTS_DIR, exist_ok=True)

# OUTPUT_FILE = os.path.join(AQI_RESULTS_DIR, "aqi_predictions.csv")

# # NEW AQI RESULTS FOLDER
# AQI_RESULTS_DIR = os.path.join(DATA_DIR, "aqi_calculation_results")
# os.makedirs(AQI_RESULTS_DIR, exist_ok=True)

# OUTPUT_FILE = os.path.join(AQI_RESULTS_DIR, "aqi_predictions.csv")
# # -------------------------------
# # CPCB BREAKPOINTS
# # -------------------------------
# BREAKPOINTS = {
#     "pm25": [(0,30,0,50),(31,60,51,100),(61,90,101,200),
#              (91,120,201,300),(121,250,301,400),(251,500,401,500)],
#     "pm10": [(0,50,0,50),(51,100,51,100),(101,250,101,200),
#              (251,350,201,300),(351,430,301,400),(431,600,401,500)],
#     "no2":  [(0,40,0,50),(41,80,51,100),(81,180,101,200),
#              (181,280,201,300),(281,400,301,400),(401,600,401,500)],
#     "so2":  [(0,40,0,50),(41,80,51,100),(81,380,101,200),
#              (381,800,201,300),(801,1600,301,400),(1601,2000,401,500)],
#     "o3":   [(0,50,0,50),(51,100,51,100),(101,168,101,200),
#              (169,208,201,300),(209,748,301,400),(749,1000,401,500)],
#     "co":   [(0,1,0,50),(1.1,2,51,100),(2.1,10,101,200),
#              (10.1,17,201,300),(17.1,34,301,400),(34.1,50,401,500)]
# }

# # -------------------------------
# # AQI SUB-INDEX FUNCTION
# # -------------------------------
# def calculate_subindex(cp, breakpoints):
#     for bp_low, bp_high, i_low, i_high in breakpoints:
#         if bp_low <= cp <= bp_high:
#             return ((i_high - i_low) / (bp_high - bp_low)) * (cp - bp_low) + i_low
#     return None

# # -------------------------------
# # AQI CATEGORY FUNCTION
# # -------------------------------
# def aqi_category(aqi):
#     if aqi <= 50:
#         return "Good"
#     elif aqi <= 100:
#         return "Satisfactory"
#     elif aqi <= 200:
#         return "Moderate"
#     elif aqi <= 300:
#         return "Poor"
#     elif aqi <= 400:
#         return "Very Poor"
#     else:
#         return "Severe"

# # -------------------------------
# # LOAD ALL PREDICTIONS
# # -------------------------------
# dfs = []

# for pollutant in BREAKPOINTS.keys():
#     file_path = os.path.join(PRED_DIR, f"{pollutant}_predictions.csv")
#     df = pd.read_csv(file_path)
#     df = df.rename(columns={f"{pollutant}_pred": pollutant})
#     dfs.append(df)

# # Merge all pollutants on date
# aqi_df = dfs[0]
# for df in dfs[1:]:
#     aqi_df = aqi_df.merge(df, on="date")

# # -------------------------------
# # CALCULATE SUB-INDICES
# # -------------------------------
# for pollutant, bps in BREAKPOINTS.items():
#     aqi_df[f"{pollutant}_sub"] = aqi_df[pollutant].apply(
#         lambda x: calculate_subindex(x, bps)
#     )

# # -------------------------------
# # FINAL AQI
# # -------------------------------
# sub_cols = [col for col in aqi_df.columns if col.endswith("_sub")]
# aqi_df["AQI"] = aqi_df[sub_cols].max(axis=1)
# aqi_df["AQI_Category"] = aqi_df["AQI"].apply(aqi_category)

# # -------------------------------
# # SAVE OUTPUT
# # -------------------------------
# aqi_df.to_csv(OUTPUT_FILE, index=False)
# print(f"✅ AQI predictions saved to: {OUTPUT_FILE}")


In [13]:
# import os
# import pandas as pd

# # -------------------------------
# # PATHS (FIXED)
# # -------------------------------
# # Always point to AQI root folder
# PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))

# DATA_DIR = os.path.join(PROJECT_ROOT, "data")
# PRED_DIR = os.path.join(DATA_DIR, "predictions")

# AQI_RESULTS_DIR = os.path.join(DATA_DIR, "aqi_calculation_results")
# os.makedirs(AQI_RESULTS_DIR, exist_ok=True)

# OUTPUT_FILE = os.path.join(AQI_RESULTS_DIR, "aqi_predictions.csv")

# # -------------------------------
# # CPCB BREAKPOINTS
# # -------------------------------
# BREAKPOINTS = {
#     "pm25": [(0,30,0,50),(31,60,51,100),(61,90,101,200),
#              (91,120,201,300),(121,250,301,400),(251,500,401,500)],
#     "pm10": [(0,50,0,50),(51,100,51,100),(101,250,101,200),
#              (251,350,201,300),(351,430,301,400),(431,600,401,500)],
#     "no2":  [(0,40,0,50),(41,80,51,100),(81,180,101,200),
#              (181,280,201,300),(281,400,301,400),(401,600,401,500)],
#     "so2":  [(0,40,0,50),(41,80,51,100),(81,380,101,200),
#              (381,800,201,300),(801,1600,301,400),(1601,2000,401,500)],
#     "o3":   [(0,50,0,50),(51,100,51,100),(101,168,101,200),
#              (169,208,201,300),(209,748,301,400),(749,1000,401,500)],
#     "co":   [(0,1,0,50),(1.1,2,51,100),(2.1,10,101,200),
#              (10.1,17,201,300),(17.1,34,301,400),(34.1,50,401,500)]
# }

# # -------------------------------
# # AQI FUNCTIONS
# # -------------------------------
# def calculate_subindex(cp, breakpoints):
#     for bp_low, bp_high, i_low, i_high in breakpoints:
#         if bp_low <= cp <= bp_high:
#             return ((i_high - i_low) / (bp_high - bp_low)) * (cp - bp_low) + i_low
#     return None

# def aqi_category(aqi):
#     if aqi <= 50: return "Good"
#     elif aqi <= 100: return "Satisfactory"
#     elif aqi <= 200: return "Moderate"
#     elif aqi <= 300: return "Poor"
#     elif aqi <= 400: return "Very Poor"
#     else: return "Severe"

# # -------------------------------
# # LOAD & MERGE PREDICTIONS
# # -------------------------------
# dfs = []

# for pollutant in BREAKPOINTS.keys():
#     file_path = os.path.join(PRED_DIR, f"{pollutant}_predictions.csv")

#     if not os.path.exists(file_path):
#         print(f"⚠ Missing file skipped: {file_path}")
#         continue

#     df = pd.read_csv(file_path)
#     df = df.rename(columns={f"{pollutant}_pred": pollutant})
#     dfs.append(df)

# if not dfs:
#     raise RuntimeError("❌ No prediction files found!")

# aqi_df = dfs[0]
# for df in dfs[1:]:
#     aqi_df = aqi_df.merge(df, on="date")

# # -------------------------------
# # SUB-INDICES & FINAL AQI
# # -------------------------------
# for pollutant, bps in BREAKPOINTS.items():
#     aqi_df[f"{pollutant}_sub"] = aqi_df[pollutant].apply(
#         lambda x: calculate_subindex(x, bps)
#     )

# sub_cols = [c for c in aqi_df.columns if c.endswith("_sub")]
# aqi_df["AQI"] = aqi_df[sub_cols].max(axis=1)
# aqi_df["AQI_Category"] = aqi_df["AQI"].apply(aqi_category)

# # -------------------------------
# # SAVE OUTPUT
# # -------------------------------
# aqi_df.to_csv(OUTPUT_FILE, index=False)
# print("✅ AQI calculation completed")
# print(f"📁 Saved at: {OUTPUT_FILE}")


In [14]:
# import os
# import pandas as pd

# # -------------------------------
# # PATHS (CORRECTED)
# # -------------------------------
# # Script runs from: AQI/3-AQI Computation (CPCB Formula)
# PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))

# DATA_DIR = os.path.join(PROJECT_ROOT, "data")
# PRED_DIR = os.path.join(DATA_DIR, "predictions")

# AQI_RESULTS_DIR = os.path.join(DATA_DIR, "aqi_calculation_results")
# os.makedirs(AQI_RESULTS_DIR, exist_ok=True)

# OUTPUT_FILE = os.path.join(AQI_RESULTS_DIR, "aqi_predictions.csv")

# # -------------------------------
# # CPCB BREAKPOINTS
# # -------------------------------
# BREAKPOINTS = {
#     "pm25": [(0,30,0,50),(31,60,51,100),(61,90,101,200),
#              (91,120,201,300),(121,250,301,400),(251,500,401,500)],
#     "pm10": [(0,50,0,50),(51,100,51,100),(101,250,101,200),
#              (251,350,201,300),(351,430,301,400),(431,600,401,500)],
#     "no2":  [(0,40,0,50),(41,80,51,100),(81,180,101,200),
#              (181,280,201,300),(281,400,301,400),(401,600,401,500)],
#     "so2":  [(0,40,0,50),(41,80,51,100),(81,380,101,200),
#              (381,800,201,300),(801,1600,301,400),(1601,2000,401,500)],
#     "o3":   [(0,50,0,50),(51,100,51,100),(101,168,101,200),
#              (169,208,201,300),(209,748,301,400),(749,1000,401,500)],
#     "co":   [(0,1,0,50),(1.1,2,51,100),(2.1,10,101,200),
#              (10.1,17,201,300),(17.1,34,301,400),(34.1,50,401,500)]
# }

# # -------------------------------
# # AQI FUNCTIONS
# # -------------------------------
# def calculate_subindex(cp, breakpoints):
#     for bp_low, bp_high, i_low, i_high in breakpoints:
#         if bp_low <= cp <= bp_high:
#             return ((i_high - i_low) / (bp_high - bp_low)) * (cp - bp_low) + i_low
#     return None

# def aqi_category(aqi):
#     if aqi <= 50: return "Good"
#     elif aqi <= 100: return "Satisfactory"
#     elif aqi <= 200: return "Moderate"
#     elif aqi <= 300: return "Poor"
#     elif aqi <= 400: return "Very Poor"
#     else: return "Severe"

# # -------------------------------
# # LOAD & MERGE PREDICTIONS
# # -------------------------------
# dfs = []

# for pollutant in BREAKPOINTS:
#     file_path = os.path.join(PRED_DIR, f"{pollutant}_predictions.csv")

#     if not os.path.exists(file_path):
#         raise FileNotFoundError(f"❌ Missing file: {file_path}")

#     df = pd.read_csv(file_path)

#     # Fix timezone issue
#     df["date"] = pd.to_datetime(df["date"]).dt.tz_localize(None)

#     df = df.rename(columns={f"{pollutant}_pred": pollutant})
#     dfs.append(df)

# aqi_df = dfs[0]
# for df in dfs[1:]:
#     aqi_df = aqi_df.merge(df, on="date", how="inner")

# # -------------------------------
# # SUB-INDICES & FINAL AQI
# # -------------------------------
# for pollutant, bps in BREAKPOINTS.items():
#     aqi_df[f"{pollutant}_sub"] = aqi_df[pollutant].apply(
#         lambda x: calculate_subindex(x, bps)
#     )

# sub_cols = [c for c in aqi_df.columns if c.endswith("_sub")]
# aqi_df["AQI"] = aqi_df[sub_cols].max(axis=1)
# aqi_df["AQI_Category"] = aqi_df["AQI"].apply(aqi_category)

# # -------------------------------
# # SAVE OUTPUT
# # -------------------------------
# aqi_df.to_csv(OUTPUT_FILE, index=False)
# print("🎉 AQI calculation completed successfully")
# print(f"📁 Saved at: {OUTPUT_FILE}")


🎉 AQI calculation completed successfully
📁 Saved at: C:\Users\parmi\CDAC_DBDA_PROJECT\AQI\data\aqi_calculation_results\aqi_predictions.csv


In [9]:
print(len(aqi_df))


0
